In [5]:
###Convert Dicom2Nifti
import os, datetime, csv
import subprocess
from itertools import islice
import pandas as pd
import pydicom
import nibabel as nib
import dicom2nifti

def convert_dcm_to_nii(dcm_dir, nii_out_path):
    """Converts DICOM files in a directory to a NIfTI file."""
    dicom2nifti.convert_directory(dcm_dir, nii_out_path, compression=True, reorient=True)

data = pd.read_csv('.csv', sep = ",")
df = pd.DataFrame(data)

destination = ''

for index, row in df.iloc[0:1].iterrows(): 
    #print(row)
    pid = row['PatientID']
    studyDate = row['StudyDate']

    # Convert the string to a datetime object
    date_object = datetime.datetime.strptime(studyDate, "%Y-%m-%d")
    date = date_object.strftime('%Y%m%d')
    
    studyId = row['StudyInstanceUID']
    seriesId = row['DTI_SeriesInstanceUID']
  
    path2dcm = os.path.join(destination, str(pid), 'dicom',  date, studyId, seriesId)
    path2nii = os.path.join(destination, str(pid), 'nifti')

    convert_dcm_to_nii(path2dcm, path2nii)   

In [6]:
import SimpleITK as sitk
import glob

data = pd.read_csv('.csv', sep = ",")
df = pd.DataFrame(data)

destination = ''

for index, row in df.iloc[0:1].iterrows(): 
    pid = row['PatientID']
    print(pid)

    #os.remove(os.path.join(destination, str(pid), 'nifti', 'dwi.nii.gz'))
    filename = '*_dwi.nii.gz'
    filepath = glob.glob(os.path.join(destination, str(pid), 'nifti', filename ))

    # Load the image
    image = sitk.ReadImage(filepath)

    # Get the size of the image
    size = image.GetSize()

    # Iterate over the 4th dimension (time)
    for t in range(size[3]):
        # Extract the 3D volume at time t
        volume = image[:, :, :, t]

        # Do something with the volume, e.g., save it
        dwi_out = os.path.join(destination, str(pid), 'nifti', f'dwi_{t}.nii.gz')
        sitk.WriteImage(volume, dwi_out)

1008684
